In [ ]:

from retinaface.pre_trained_models import get_model
import torch
import cv2
import numpy as np
retina_face_model = get_model("resnet50_2020-07-20", max_size=2048)
retina_face_model.eval() 
import numpy as np
import os
import torch
import matplotlib.pyplot as plt
device = torch.device('cuda:0' if torch.cuda.is_available() else 'cpu')
resnet_model = torch.load('resnet_model.pt')
from numpy.linalg import norm
import os

from torch.nn import functional as F

def get_embedding(face_pixels):
    face_pixels = face_pixels.astype('float32')
    mean, std = face_pixels.mean(), face_pixels.std()
    face_pixels = (face_pixels - mean) / std
    resized1 = torch.from_numpy(face_pixels)
    resized1= resized1.permute(2,0,1)
    img_cropped = torch.unsqueeze(resized1, dim=0)
    img_embedding  = resnet_model(img_cropped)
    return img_embedding.detach().numpy()

def calculate_sim_score(image_embed1,image_embed2) -> float:
    """Calclulate Similarity Scores from Image embedding 
    bytes datatype to ndarray format.
    Args:
        image_embed1 (tensor(1,512)): image embedding for image 1
        image_embed2 (tensor(1,512)): image embedding for image 2
    Returns:
        sim_score  (float): Similarity Socre
    """
    sim_scores = F.cosine_similarity(image_embed1,image_embed2).detach().numpy()
    sim_score = np.clip(sim_scores,a_min=0.0,a_max=1.0)[0]
    return sim_score

def fn_return_sim_score(img_embedding):
    write_flag = False
    ecu_dists = np.array([])
    sim_scores = np.array([])
    id = None
    sim_score = 0
    ecu_dist = 0
    for file in os.listdir(r'anchor_embedding'):
        # embedding1.npy
        embedding1 = np.load('anchor_embedding/'+file)
        img_embedding = img_embedding.reshape(-1)
        embedding1 = embedding1.reshape(-1)
        # print(img_embedding.shape,embedding1.shape)
        cosine_sim = np.dot(img_embedding,embedding1)/(norm(img_embedding)*norm(embedding1))
        dist = np.linalg.norm(img_embedding-embedding1)        
        # img_embedding1 = np.load(r'anchor_embedding/'+file)
        # img_embedding1 = torch.from_numpy(img_embedding1)
        # sim_score = calculate_sim_score(img_embedding1,img_embedding)
        # sim_score = sim_score * 100
        sim_scores = np.append(sim_scores,cosine_sim)
        ecu_dists = np.append(ecu_dists,dist)
        # print(cosine_sim,dist)
    if len(ecu_dists) != 0:
        id = np.argmin(ecu_dists) 
        ecu_dist = ecu_dists[id] 
    if len(sim_scores) != 0:
        id = np.argmax(sim_scores) 
        sim_score = sim_scores[id] * 100 
        id = id + 1       
    if sim_score < 60:
        write_flag = True
    print(id,ecu_dist,sim_score)
    return ecu_dist,write_flag,id,sim_score

def getFrame(sec,vidcap,detection_model,out,count):
    vidcap.set(cv2.CAP_PROP_POS_MSEC,sec*1000)
    hasFrames,frame = vidcap.read()
    resized = None
    break_flag = None
    cropped_image = None
    sim_score = 0
    ecu_dist = 0
    if hasFrames:
        image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
        annotations = retina_face_model.predict_jsons(image)
        color = (255, 0, 0)
        for annotation in annotations:
            prob  = annotation['score'] * 100
            margin = 5
            if annotation["score"] != -1:
                x_min, y_min, x_max, y_max = annotation["bbox"]
                box_area = (int(x_max) - int(x_min)) * (int(y_max) - int(y_min))                    
                if box_area > 5000:                  
                    x_min = np.clip(x_min, 0, x_max - 1)
                    y_min = np.clip(y_min, 0, y_max - 1)
                    box_area = (int(x_max) - int(x_min)) * (int(y_max) - int(y_min))
                    bb = np.array([int(x_min) ,int(y_min),int(x_max),int(y_max)], dtype=np.int32)           
                    cropped = image[bb[1]:bb[3], bb[0]:bb[2], :].copy()
                    frame = cv2.rectangle(frame,(int(x_min),int(y_min)),(int(x_max),int(y_max)),(255, 0, 0) ,4)
                    landmarks_cropped = np.array(annotation['landmarks'])
                    leftEyeCenter = np.array(landmarks_cropped[0])
                    rightEyeCenter = np.array(landmarks_cropped[1])
                    dY = rightEyeCenter[1] - leftEyeCenter[1]
                    dX = rightEyeCenter[0] - leftEyeCenter[0]
                    angle = np.degrees(np.arctan2(dY, dX)) 
                    # print(angle)
                    dist = np.sqrt((dX ** 2) + (dY ** 2))
                    # desiredDist = 0.4
                    # desiredDist *= desiredFaceWidth
                    scale = 1
                    height, width = cropped.shape[:2]
# get the center coordinates of the 
# image to create the 2D rotation matrix
                    center = (width/2, height/2)
                    M = cv2.getRotationMatrix2D(center, angle, scale)

                    x1,y1 = np.dot(M,np.array([bb[0],bb[1],1]).T)
                    x2,y2 = np.dot(M,np.array([bb[2],bb[3],1]).T)
                    # print((x_max - x_min))
                    aspect_ratio = (y2 - y1)/(x2 - x1)
                    margin = 35
                    print(aspect_ratio)
                    if aspect_ratio > 1.5:
                        x1 = int(max(0,x1-(margin * aspect_ratio )))
                        x2 = int(x2+(margin *aspect_ratio ))

                    # M[1, 2] += (tY - eyesCenter[1]) 
                    h,w = image.shape[:2]

                    output = cv2.warpAffine(image, M, (w, w),
                        flags=cv2.INTER_CUBIC)  
                    cropped = output.copy()
                    cropped = cropped[int(y1):int(y2),int(x1):int(x2),:]
                    output = cv2.rectangle(output,(int(x1),int(y1)),(int(x2),int(y2)),(255, 0, 0) ,4)
                    cv2.imwrite("aligned_face_img"+str(count)+".jpg", output) 
                    count = count + 1                           
                    if not os.path.exists('anchor_embedding'):
                        os.makedirs('anchor_embedding')     
                    if not os.path.exists('anchor_images'):
                        os.makedirs('anchor_images')                                                                       
                    resized = cv2.resize(cropped, (160,160), interpolation = cv2.INTER_AREA)
                    embeddings_list = os.listdir(r'anchor_embedding') 
                    img_embedding = get_embedding(resized)
                    if len(embeddings_list) != 0:                    
                        ecu_dist,write_flag,id,sim_score = fn_return_sim_score(img_embedding)
                    else:
                        write_flag = True  
                    if write_flag == True:
                        id = len(embeddings_list) + 1
                        np.save(r"anchor_embedding/"+'embedding'+str(id),img_embedding)                    
                        cropped = cv2.cvtColor(cropped, cv2.COLOR_RGB2BGR)
                        cv2.imwrite(r"anchor_images/"+"image"+str(id)+".jpg", cropped)  
                        # print(annotation["bbox"],annotation["score"],ecu_dist,sim_score)                     
                    if write_flag == False:
                        frame = cv2.putText(frame, 'Similarity score ' + str(int(sim_score)) + '  Id ' + str(id) , (int(x_min),int(y_min)-10), cv2.FONT_HERSHEY_SIMPLEX, 0.9, (36,255,12), 2)                        
                        # print(annotation["bbox"],annotation["score"],ecu_dist,sim_score,str(id))
                    out.write(frame)
    return hasFrames,break_flag,count

                

vidcap = cv2.VideoCapture('Webcam_test.mov')
sec = 0
frameRate = 0.5 #//it will capture image in each 0.5 second
count=1
frame_width = int(vidcap.get(3))
frame_height = int(vidcap.get(4))

detection_model = 'retinaface'
fourcc = cv2.VideoWriter_fourcc(*'MP4V')
out = cv2.VideoWriter('webcam_test_op.mp4', fourcc, 20.0, (frame_width,frame_height))
count = 0
hasFrames,break_flag,count = getFrame(sec,vidcap,detection_model,out,count)

while hasFrames:
    # count = count + 1
    sec = sec + frameRate
    sec = round(sec, 2)
    hasFrames,break_flag,count = getFrame(sec,vidcap,detection_model,out,count)
    if break_flag == True:
        break

vidcap.release()
out.release()  